### Tutorial
**Read and reconstruct LDCT-projections with Pyxu.**
*Prerequisites*: ensure that you have the `pyxu-ldct-reader` package installed in your Python environment. You can install it using the following command:

In [ ]:
!pip install pyxu-ldct-reader

### Load data
To reproduce this example, download the data contained in the TCIA Low Dose CT Image and Projection Data (LDCT-and-Projection-data) repository (https://doi.org/10.7937/9npb-2637).

In [ ]:
import numpy as np

# Parse dataset from dicom files, where:
# proj_data is the projection data, given as the line integral of the linear attenuation coefficient (g/cm^3). Its unit is thus g/cm^2.
# n_spec are ray directions for each projection.
# t_spec are the offset specifications for each projection.
from pyxu_ldct_reader.reader import load_projections
folder = '/path/to/LDCT-and-Projection-data/C001/01-31-2022-NA-NA-49153/1.000000-Full Dose Projections-31861/'
proj_data, n_spec, t_spec = load_projections(folder)

# Use only 10% of the original dataset to have faster results.
percentage_data = 10
ids = np.arange(int(len(proj_data) * percentage_data/100))
z_dim = int(314 * percentage_data/100)
proj_data = proj_data[ids]
n_spec = n_spec[ids]
t_spec = t_spec[ids]

# Print dataset shape
print(proj_data.shape, proj_data.dtype)
print(n_spec.shape, n_spec.dtype)
print(t_spec.shape, t_spec.dtype)

### Define volume
Shape and voxel size.

In [ ]:
# Define reconstruction volume shape
dim_shape = (512, 512, z_dim)

# Define the voxel size 
pitch =[0.736328, 0.736328, 1.25]

### Select hardware

In [ ]:
# If you have a GPU, you can accelerate the reconstruction
GPU = False    
if GPU:
    import cupy as cp
    proj_data = cp.asarray(proj_data)
    n_spec = cp.asarray(n_spec)
    t_spec = cp.asarray(t_spec)

### Define the forward model (X-Ray transform) based on the geometry

In [ ]:
# Shift to offset convention
offset = pitch * np.array(dim_shape)[np.newaxis]/2

# Instantiate XRayTransform
from pyxu.operator.linop.xrt.ray import RayXRT
xray_transform = RayXRT.init(
    dim_shape=dim_shape,
    pitch=pitch,
    n_spec=n_spec.reshape(-1, 3),
    t_spec=t_spec.reshape(-1, 3) + offset, 
)

### Run backward projection (BP)

In [ ]:
# Run backward projection at single (float32) precision.
import pyxu.runtime as pxrt
with pxrt.Precision(pxrt.Width.SINGLE):
    bp = xray_transform.adjoint(proj_data.ravel()).reshape(dim_shape)

### Run pseudo-inverse (PINV)

In [ ]:
# Define a stopping criteria
import pyxu.opt.stop as pxst
stop_crit = pxst.RelError(eps=1e-4, var="x", f=None, norm=2, satisfy_all=True) | pxst.MaxIter(200)
    
# Run optimization at single (float32) precision.
import pyxu.runtime as pxrt
with pxrt.Precision(pxrt.Width.SINGLE):
    pinv = xray_transform.pinv(
            proj_data.ravel(), 
            damp=10,
            kwargs_init=dict(verbosity=10), 
            kwargs_fit=dict(stop_crit=stop_crit),
    ).reshape(dim_shape)

### Save outputs

In [ ]:
import tifffile
if GPU:
    bp = bp.get()
    pinv = pinv.get()
tifffile.imwrite('bp.tif', bp.transpose(2, 1, 0))
tifffile.imwrite('pinv.tif', pinv.transpose(2, 1, 0))